In [29]:
import torch.nn.functional as F

In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
from pathlib import Path
from torch.nn import Module
logging.basicConfig(level=logging.ERROR)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
root_dir = Path('.')
data_dir = Path(root_dir,'.data', 'sentence-classification')

In [58]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, is_train):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Phrase
        self.is_train = is_train
        if self.is_train:
            self.targets = self.data.Sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        if self.is_train:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.float)
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            }

def get_data_loader(df, tokenizer, max_len, batch_size, is_train, shuffle):
    dataset = SentimentData(
        df,
        tokenizer = tokenizer,
        max_len = max_len,
        is_train=is_train,
    )
    
    return DataLoader(
        dataset,
        shuffle = shuffle,
        batch_size=batch_size
    )

def get_predictions(model, loader):
    model = model.eval()
    
    predictions = []
    predictions_probs = []
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['ids']
            attention_mask = batch['mask']
            token_type_ids = batch["token_type_ids"]
            
            if torch.cuda.is_available():
                input_ids = input_ids.cuda()
                attention_mask = attention_mask.cuda()
                token_type_ids = token_type_ids.cuda()
                
            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                token_type_ids = token_type_ids
            )
                        
            predictions.extend(outputs.detach().cpu().numpy())
    predictions = np.array(predictions)
    predictions = torch.from_numpy(predictions)
    return F.softmax(predictions,dim=1)

In [4]:
# Defining some key variables that will be used later on in the training
load_model = False
if load_model:
    #이거 왜 안되지..
#     tokenizer = RobertaTokenizer.from_pretrained('./vocab.json',truncation=True, do_lower_case=True)
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
else:
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

max_len = 256
batch_size = 8
# EPOCHS = 1
LEARNING_RATE = 1e-05
train_valid_frac = 0.8
#원래 데이터셋
train = pd.read_csv(data_dir.joinpath('train_plus.csv'))
new_df = train[['Phrase', 'Sentiment']]

train_df=new_df.sample(frac=train_valid_frac,random_state=200)
valid_df=new_df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

test_df = pd.read_csv(data_dir.joinpath('eval_final_open.csv'))
test_df = test_df[['Sentence']]
test_df.rename(columns = {'Sentence': 'Phrase'}, inplace = True)



print(f'Dataset Configuration')
print(f'-'*25)
print(f'Train/Valid = {train_valid_frac:.2f}/{1-train_valid_frac:.2f}')
print(f'Batch size = {batch_size}')
print(f'-'*25)
print(f'Train set : {len(train_df)}')
print(f'Valid set : {len(valid_df)}')
print(f'Test set : {len(test_df)}')

training_loader   = get_data_loader(new_df, tokenizer, max_len, batch_size, True, True)
validating_loader = get_data_loader(valid_df, tokenizer, max_len, batch_size, True, True)
testing_loader    = get_data_loader(test_df, tokenizer, max_len, batch_size, False, False)


# training_loader   = DataLoader(training_set, **train_params)
# validating_loader = DataLoader(validating_set, **valid_params)
# testing_loader    = DataLoader(testing_set, **test_params)

Dataset Configuration
-------------------------
Train/Valid = 0.80/0.20
Batch size = 8
-------------------------
Train set : 69809
Valid set : 17452
Test set : 4311


In [5]:
class ActiveDropout(Module):
    # all building blocks of networks are inherited from Module!

    def __init__(self, p=0.5):
        super().__init__()  # init the base class
        self.p = p

    def forward(self, input):
        mask = torch.rand_like(input) > self.p
        return input * mask.to(input) / (1 - self.p)
    
        pass

In [6]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier1 = torch.nn.Linear(768, 768)
        self.pre_classifier2 = torch.nn.Linear(768, 768)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        
        pooler = self.pre_classifier1(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = ActiveDropout(0.5)(pooler)
        
        pooler = self.pre_classifier2(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = ActiveDropout(0.5)(pooler)
        
        output = self.classifier(pooler)
        return output

In [7]:
model = RobertaClass()
if load_model:
    model = torch.load('pytorch_roberta_sentiment_plus_bay.bin')
model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, eleme

In [8]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0 = 5,
    T_mult = 1,
    eta_min = 2e-7
)

In [9]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [10]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch,data_loader):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(data_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
#         print(ids.shape)
#         print(mask.shape)
#         print(token_type_ids.shape)
        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%500==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        scheduler.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [11]:
EPOCHS = 20
for epoch in range(EPOCHS):
    train(epoch,training_loader)

1it [00:00,  1.05it/s]

Training Loss per 5000 steps: 1.6148021221160889
Training Accuracy per 5000 steps: 25.0


501it [01:16,  6.25it/s]

Training Loss per 5000 steps: 1.4953281741418287
Training Accuracy per 5000 steps: 32.734530938123754


1001it [02:33,  6.50it/s]

...

Training Loss per 5000 steps: 0.21679871612840015
Training Accuracy per 5000 steps: 91.21712828717128


10501it [25:29,  6.88it/s]

Training Loss per 5000 steps: 0.21632047522419945
Training Accuracy per 5000 steps: 91.23297781163699


10908it [26:28,  6.87it/s]

The Total Accuracy for Epoch 19: 91.19996332840559
Training Loss Epoch: 0.21691931191361746
Training Accuracy Epoch: 91.19996332840559


In [13]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(validating_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [14]:
valid(model,testing_loader)

3it [00:00, 22.21it/s]

Validation Loss per 100 steps: 0.032554991543293
Validation Accuracy per 100 steps: 100.0


105it [00:04, 23.35it/s]

Validation Loss per 100 steps: 0.09085716966926904
Validation Accuracy per 100 steps: 96.41089108910892


163it [00:07, 23.20it/s]


KeyboardInterrupt: 

In [83]:
def MC_soft_predict(model,testing_loader,MC_num):
    MC_pred = get_predictions(model, testing_loader)
    for i in range(MC_num-1):
        predictions = get_predictions(model, testing_loader)
        MC_pred     += predictions
    MC_pred = MC_pred/MC_num
    result = torch.argmax(MC_pred, dim=1)
    resulti = result.numpy()
    
    return result

In [84]:
result = MC_soft_predict(model,testing_loader,100)

In [85]:
submission = pd.DataFrame({'Id' : range(len(result)), 'Category' : result})
submission.to_csv('submission.csv', index=False)

In [12]:
output_model_file = 'pytorch_roberta_sentiment_plus_bay_sche_reg_layerdiff.bin'
output_vocab_file = './'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


# train dil

In [12]:
dil_iter = 5
MC_iter  = 100
EPOCHS = 10

result = MC_predict(model,testing_loader,MC_iter)
for i in range(dil_iter):
    result_series = pd.Series(result)
    dil_train = pd.DataFrame(test_df["Phrase"])
    dil_train["Sentiment"] = result_series
    
    dil_loader   = get_data_loader(dil_train, tokenizer, max_len, batch_size, True, True)
    
    for epoch in range(EPOCHS):
        train(epoch,dil_loader)
    
    result = MC_predict(model,testing_loader,MC_iter)
    
    submission = pd.DataFrame({'Id' : range(len(result)), 'Category' : result})
    submission.to_csv('submission_T'+str(i)+'.csv', index=False)

0it [00:00, ?it/s]

pred_np의 shape :  (100, 4311)
pred_mean의 shape :  (4311,)
Training Loss per 5000 steps: 0.34587007761001587
Training Accuracy per 5000 steps: 75.0


501it [01:14,  6.51it/s]

Training Loss per 5000 steps: 0.42064276596386274
Training Accuracy per 5000 steps: 84.03193612774452


539it [01:20,  6.69it/s]
1it [00:00,  6.55it/s]

...

Training Loss per 5000 steps: 0.022687560444039096
Training Accuracy per 5000 steps: 99.47604790419162


539it [01:18,  6.84it/s]
1it [00:00,  7.07it/s]

The Total Accuracy for Epoch 4: 99.48967756900952
Training Loss Epoch: 0.02167683300559071
Training Accuracy Epoch: 99.48967756900952
Training Loss per 5000 steps: 0.0006244438118301332
Training Accuracy per 5000 steps: 100.0


152it [00:22,  6.78it/s]


KeyboardInterrupt: 